In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection as  model_selection
#load the dataset
from sklearn.datasets import fetch_california_housing

# Load the diabetes dataset
data_X, data_y = fetch_california_housing(return_X_y=True)

#remember index starts from 0
# Split the data into training/testing sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(data_X, data_y, test_size=0.2, random_state=0)

X_train, X_val, y_train, y_val  = model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=0)

print("Total dataset elements",data_X.shape)
print("Train dataset elements",X_train.shape)
print("Validatiom dataset elements",X_val.shape)
print("Test dataset elements",X_test.shape)

Total dataset elements (20640, 8)
Train dataset elements (12384, 8)
Validatiom dataset elements (4128, 8)
Test dataset elements (4128, 8)


In [3]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The intercept
print("Coefficients: \n", regr.intercept_)

y_pred = regr.predict(X_test)



# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

#see the impact of each variable

regr_coef=regr.coef_/np.max(regr.coef_)
print("Coefficients: \n", regr_coef)
#Here, value=1 is the most relevant feature.
california_housing = fetch_california_housing(as_frame=True)
california_housing.data.head()

Coefficients: 
 [ 4.52681080e-01  9.44835024e-03 -1.30201996e-01  8.24878307e-01
 -5.84727391e-06 -7.62397662e-03 -4.06305340e-01 -4.18414937e-01]
Coefficients: 
 -35.66840017162287
Mean squared error: 0.53
Coefficient of determination: 0.59
Coefficients: 
 [ 5.48785288e-01  1.14542353e-02 -1.57843885e-01  1.00000000e+00
 -7.08865036e-06 -9.24254711e-03 -4.92563977e-01 -5.07244443e-01]


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [8]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm

# Assuming you have already fitted the linear regression model (regr.fit(X_train, y_train))

# Add a constant column to the feature matrix (required by statsmodels)
X_train_with_constant = sm.add_constant(X_train)

# Fit the OLS (Ordinary Least Squares) model
ols_model = sm.OLS(y_train, X_train_with_constant).fit()

# Get summary statistics of the model
summary = ols_model.summary()

# Extract p-values from the summary for all features
p_values = summary.tables[1].data[1:]

# Create a DataFrame to associate p-values with feature names
p_values_df = pd.DataFrame(p_values, columns=['Feature', 'Coefficient', 'Standard Error', 't-value', 'P-Value', 'Lower CI', 'Upper CI'])
p_values_df['P-Value'] = p_values_df['P-Value'].astype(float)


print(summary)
print(p_values_df)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     2461.
Date:                Sun, 13 Aug 2023   Prob (F-statistic):               0.00
Time:                        16:18:07   Log-Likelihood:                -13464.
No. Observations:               12384   AIC:                         2.695e+04
Df Residuals:                   12375   BIC:                         2.701e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -35.6684      0.849    -42.006      0.0

In [9]:
from scipy import stats
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)
params = np.append(lm.intercept_,lm.coef_)
predictions = lm.predict(X_train)

newX = pd.DataFrame({"Constant":np.ones(len(X_train))}).join(pd.DataFrame(X_train))
MSE = (sum((y_train-predictions)**2))/(len(newX)-len(newX.columns))

# Note if you don't want to use a DataFrame replace the two lines above with
# newX = np.append(np.ones((len(X),1)), X, axis=1)
# MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/ sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX[0])))) for i in ts_b]

sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)

myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilities"] = [params,sd_b,ts_b,p_values]
print(myDF3)

   Coefficients  Standard Errors  t values  Probabilities
0      -35.6684            0.849   -42.006            NaN
1        0.4527            0.006    81.204            NaN
2        0.0094            0.001    16.493            NaN
3       -0.1302            0.008   -16.185            NaN
4        0.8249            0.041    20.252            NaN
5       -0.0000            0.000    -0.951            NaN
6       -0.0076            0.001    -6.442            NaN
7       -0.4063            0.009   -43.858            NaN
8       -0.4184            0.010   -43.176            NaN
